### ETL for Loading Downloaded Dataset to MySQL Database

In [3]:
# Loading csv files from my folders into pandas dataframe
import pandas as pd
import os

# Specific folder address
folder_address = r"C:\Users\alank\Documents\Data Analyst Projects\Azure End to End Data Anlaytics Project\olist dataset"

# List all CSV files in the specified folder
csv_files = [os.path.join(folder_address, file) for file in os.listdir(folder_address) if file.endswith(".csv")]

# Read each CSV file into a DataFrame
dataframes = []
for filename in csv_files:
    dataframe = pd.read_csv(filename)
    dataframes.append(dataframe)


In [7]:
# extracting the csv file names into a list
csv_filenames = [os.path.basename(filename) for filename in csv_files]
csv_filenames

['olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

In [9]:
!pip install pymysql

   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ------------------------------------ --- 41.0/44.8 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 44.8/44.8 kB 1.1 MB/s eta 0:00:00


In [10]:
import pymysql

In [11]:
db = pymysql.connect(host="localhost", user="root", password="password", database="olist_db")


In [20]:
# Create tables for each CSV file

for filename in csv_filenames:
    # Extract table name from filename
    table_name = filename.split(".")[0]

    # Read CSV data into a DataFrame
    dataframe = pd.read_csv(filename)

    # Prepare SQL query to create the table
    sql_query = "CREATE TABLE `" + table_name + "` ("

    # Construct column definitions based on DataFrame column names and data types
    column_definitions = []
    for column_name, data_type in zip(dataframe.columns, dataframe.dtypes):
        if pd.api.types.is_string_dtype(data_type):
            column_definition = "`" + column_name + "` VARCHAR(255) NOT NULL"
        elif pd.api.types.is_numeric_dtype(data_type):
            column_definition = "`" + column_name + "` INT NOT NULL"
        elif pd.api.types.is_datetime_dtype(data_type):
            column_definition = "`" + column_name + "` DATETIME NOT NULL"
        else:
            raise ValueError("Unsupported data type for column:", column_name)

        column_definitions.append(column_definition)

    # Add primary key constraint if necessary
    if "id" in dataframe.columns:
        sql_query += "PRIMARY KEY (`id`),"

    # Add remaining column definitions
    sql_query += ",".join(column_definitions) + ")";
    print(sql_query)

    # Execute SQL query to create the table
    cursor = db.cursor()
    cursor.execute(sql_query)

    # Close the cursor
    cursor.close()

# Close MySQL connection
db.close()

CREATE TABLE `olist_customers_dataset` (`customer_id` VARCHAR(255) NOT NULL,`customer_unique_id` VARCHAR(255) NOT NULL,`customer_zip_code_prefix` INT NOT NULL,`customer_city` VARCHAR(255) NOT NULL,`customer_state` VARCHAR(255) NOT NULL)
CREATE TABLE `olist_geolocation_dataset` (`geolocation_zip_code_prefix` INT NOT NULL,`geolocation_lat` INT NOT NULL,`geolocation_lng` INT NOT NULL,`geolocation_city` VARCHAR(255) NOT NULL,`geolocation_state` VARCHAR(255) NOT NULL)
CREATE TABLE `olist_orders_dataset` (`order_id` VARCHAR(255) NOT NULL,`customer_id` VARCHAR(255) NOT NULL,`order_status` VARCHAR(255) NOT NULL,`order_purchase_timestamp` VARCHAR(255) NOT NULL,`order_approved_at` VARCHAR(255) NOT NULL,`order_delivered_carrier_date` VARCHAR(255) NOT NULL,`order_delivered_customer_date` VARCHAR(255) NOT NULL,`order_estimated_delivery_date` VARCHAR(255) NOT NULL)
CREATE TABLE `olist_order_items_dataset` (`order_id` VARCHAR(255) NOT NULL,`order_item_id` INT NOT NULL,`product_id` VARCHAR(255) NOT NU

In [22]:
table_names=[]
for filename in csv_filenames:
    # Extract table name from filename
    table_name = filename.split(".")[0]
    table_names.append(table_name)

In [23]:
table_names


['olist_customers_dataset',
 'olist_geolocation_dataset',
 'olist_orders_dataset',
 'olist_order_items_dataset',
 'olist_order_payments_dataset',
 'olist_order_reviews_dataset',
 'olist_products_dataset',
 'olist_sellers_dataset',
 'product_category_name_translation']

In [32]:
from sqlalchemy import create_engine
from urllib.parse import quote

# Replace 'your_username', 'your_password', 'your_database', and 'localhost' with your actual MySQL credentials
username = 'root'
password = 'password'
host = 'localhost'
database = 'olist_db'

# Encode the password
encoded_password = quote(password)

# Create the connection string
connection_string = f"mysql+pymysql://{username}:{encoded_password}@{host}/{database}"

# Create the engine
engine = create_engine(connection_string)

# Loop to save each DataFrame to its corresponding table
for dataframe, table_name in zip(dataframes, table_names):
    dataframe.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(table_name + " loaded")

# Dispose of the engine
engine.dispose()



olist_customers_dataset loaded
olist_geolocation_dataset loaded
olist_orders_dataset loaded
olist_order_items_dataset loaded
olist_order_payments_dataset loaded
olist_order_reviews_dataset loaded
olist_products_dataset loaded
olist_sellers_dataset loaded
product_category_name_translation loaded
